In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import geopy.distance

In [10]:
fm = pd.read_csv("../full_db/2021/2021_01_01.csv", index_col=0,low_memory=False)

In [11]:
def transform_df(ene= fm):
    ene = ene.reset_index()
    ene["full_date_retiro"] = pd.to_datetime(ene["Fecha_Retiro"] + " " + ene["Hora_Retiro"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["full_date_aribo"] = pd.to_datetime(ene["Fecha_Arribo"] + " " + ene["Hora_Arribo"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["Mes"] = ene["full_date_retiro"].dt.month
    ene["Hora"] = ene["full_date_retiro"].dt.hour
    ene["time_delta"] = round((ene["full_date_aribo"]  - ene["full_date_retiro"]) / np.timedelta64(1,"m"),2)
    ene["Ciclo_Estacion_Retiro"]= ene[["Ciclo_Estacion_Retiro"]].astype(str)
    ene["Ciclo_Estacion_Retiro"] = [i[:-2] for i in ene["Ciclo_Estacion_Retiro"]]
    ene["Bici"]= ene[["Bici"]].astype(str)
    ene["Bici"] = [i[:-2] for i in ene["Bici"]]
    ene["Ciclo_Estacion_Arribo"] = ene["Ciclo_Estacion_Arribo"].astype(str)
    ene["viaje"] = ene["Ciclo_Estacion_Retiro"].astype(str)+"-"+ene["Ciclo_Estacion_Arribo"]
    ene["Genero_Usuario"] = ene["Genero_Usuario"].fillna("X")
    ene = ene.dropna(axis=0).copy()
    return ene
 
df = transform_df()

In [12]:
def estaciones_df():
   estaciones = pd.read_csv("../full_db/estaciones-de-ecobici.csv")[["id","name","districtcode","districtname","location_lat","location_lon","stationtype","punto_geo"]]
   estaciones["Ciclo_Estacion_Retiro"] = estaciones["id"].astype("str").copy()
   estaciones["Ciclo_Estacion_Arribo"] = estaciones["id"].astype("str").copy()
   estaciones_retiro = estaciones.iloc[:,[-2,1,2,3,4,5,6,7]].rename(columns={"name":"name_retiro","districtcode":"districtcode_retiro","districtname":"districtname_retiro","location_lat":"location_lat_retiro","location_lon":"location_lon_retiro","stationtype":"stationtype_retiro","punto_geo":"punto_geo_retiro"}).copy().iloc[:,[0,1,4,5,7]]
   estaciones_arribo = estaciones.iloc[:,[-1,1,2,3,4,5,6,7]].rename(columns={"name":"name_arribo","districtcode":"districtcode_arribo","districtname":"districtname_arribo","location_lat":"location_lat_arribo","location_lon":"location_lon_arribo","stationtype":"stationtype_arribo","punto_geo":"punto_geo_arribo"}).copy().iloc[:,[0,1,4,5,7]]
   return estaciones_retiro, estaciones_arribo
 
estaciones_retiro, estaciones_arribo = estaciones_df()


In [13]:
def mergingfiles(month, er, ea):
   first = month.merge(er, on="Ciclo_Estacion_Retiro", how="left").merge(ea, on="Ciclo_Estacion_Arribo", how="left")
   return first
 
exportfileII = mergingfiles(month=df, er=estaciones_retiro, ea=estaciones_arribo)

In [18]:
exportfileII

,viaje,location_lat_retiro,location_lat_retiro,location_lon_retiro,location_lat_arribo,location_lon_arribo
0,18-143,19.428880,19.428880,-99.164176,19.414303,-99.157542
1,76-76,19.408736,19.408736,-99.169913,19.408736,-99.169913
2,281-61,19.398532,19.398532,-99.157308,19.413742,-99.169525
3,405-431,19.370619,19.370619,-99.156626,19.361450,-99.158220
4,30-126,19.426442,19.426442,-99.164323,19.422288,-99.162138
...,...,...,...,...,...,...
255781,58-278,19.415680,19.415680,-99.171793,19.400710,-99.163831
255782,315-74,19.391472,19.391472,-99.173600,19.410343,-99.172831
255788,404-256,19.373813,19.373813,-99.157387,19.437200,-99.158160
255789,478-25,19.440818,19.440818,-99.176961,19.427317,-99.166662


In [24]:
def filetoexport(first, name_of_file):
   first = first[["viaje","location_lat_retiro","location_lon_retiro","location_lat_arribo","location_lon_arribo"]].replace("", np.nan).dropna(axis=0).drop_duplicates(subset="viaje")
   
   location_lat_retiro = first["location_lat_retiro"].to_list()
   location_lon_retiro = first["location_lon_retiro"].to_list()
   location_lat_arribo = first["location_lat_arribo"].to_list()
   location_lon_arribo = first["location_lon_arribo"].to_list()
 
   distances = pd.DataFrame({"location_dist":[geopy.distance.distance((location_lat_retiro[i],location_lon_retiro[i]), (location_lat_arribo[i],location_lon_arribo[i])).km for i in range(len(location_lon_arribo))]})
  
   l = pd.concat([first, distances], axis=1, join="inner")[["viaje","location_dist"]]
   l.to_csv(f"./db/distancia_por_dia//{name_of_file}.csv")

In [26]:
files = os.listdir("../full_db/2021")
estaciones_retiro, estaciones_arribo = estaciones_df()
lista_meses = [["ene",1],["feb",2],["mar",3],["apr",4],["may",5],["jun",6],["jul",7],["aug",8],["sep",9],["oct",10]]
nombre_columnas = ["Genero_Usuario","Edad_Usuario","Bici","Ciclo_Estacion_Retiro","Fecha_Retiro","Hora_Retiro","Ciclo_Estacion_Arribo","Fecha_Arribo","Hora_Arribo"]

for i,x in enumerate(files):
    jan = pd.read_csv(f"../full_db/2021/{x}", index_col=0, low_memory=False)
    df = transform_df(ene=jan)
    exportfileI = mergingfiles(month=df, er=estaciones_retiro, ea=estaciones_arribo)
    filetoexport(exportfileI, name_of_file=lista_meses[i][0])
    
    print(f"DONE {lista_meses[i][1]}")

DONE 1
DONE 2
DONE 3
DONE 4
DONE 5
DONE 6
DONE 7
DONE 8
DONE 9
DONE 10
